In [1]:
import pandas as pd
from tableschema import Schema,Table
import numpy as np
import datetime

#### Defining table schema

In [2]:
schema=Schema(
{
	"fields": [
		{
			"name": "patient_id",
			"title": "patient_id",
            "type" : "string",
			"constraints": {
				"required": True,
			}
		},
		{
			"name": "claim_id",
			"title": "claim_id",
            "type" : "number",
			"constraints": {
				"required": True,
			}
		},
		{
			"name": "diagnosis_codes",
			"title": "diagnosis_codes",
             "type" : "string",
			"constraints": {
				"required": False,
			}
		},
		{
			"name": "procedure_code",
			"title": "procedure_code",
            "type" : "number",
			"constraints": {
				"required": True,
			}
		},
		{
			"name": "date_service",
			"title": "date_service",
            "type" : "string",
			"constraints": {
				"required": True,
			}
		},
		{
			"name": "date_received",
			"title": "date_received",
            "type" : "string",
			"constraints": {
				"required": True,
			}
		}
	]
})

### Reads in csv and checks if it matches the defined schema

In [3]:
table=Table('C:/Akhil/USF BAIS/Jobs/Health Verity/sample_claims.csv',schema=schema)

### Writes any exceptions raised to an array.
#### Checks if data matches the table schema.
#### Checks for missing values in required (non null fields)
#### Checks for data type matches with table definition


In [4]:
errors = []
error_claims =[]
def exc_handler(exc, row_number=None, row_data=None, error_data=None):
    errors.append([row_number, row_data, error_data])

for row in table.iter(exc_handler=exc_handler):
    continue

for i in errors:
    #print(i[0] - 1,i[2]) # row_number -1 = claim_id
    error_claims.append(i[0] - 1)   

### Reading in csvs using pandas dataframes

In [44]:
df=pd.read_csv('C:/Akhil/USF BAIS/Jobs/Health Verity/sample_claims.csv')

In [46]:
df2.head()

,claim_id,procedure_code,date_service,date_received
0,1,99999,1/25/2021,1/26/2021
1,2,99999,1/27/2021,1/29/2021
2,3,87491,1/7/2021,1/10/2021
3,4,86735,1/15/2021,1/16/2021
4,5,83014,1/6/2021,1/7/2021


In [6]:
df_dates= df[['claim_id','date_service','date_received']]

In [7]:
df_valid_cpt_codes = pd.read_csv('C:/Akhil/USF BAIS/Jobs/Health Verity/valid_cpt_codes.csv')

In [8]:
df_valid_icd_10_codes = pd.read_csv('C:/Akhil/USF BAIS/Jobs/Health Verity/valid_icd_10_codes.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   patient_id       4994 non-null   object
 1   claim_id         5000 non-null   int64 
 2   diagnosis_codes  3714 non-null   object
 3   procedure_code   4997 non-null   object
 4   date_service     4990 non-null   object
 5   date_received    5000 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


### Dropping values from csv where claim ids in error_claims i.e. does not match schema or missing value or type mismatch

In [10]:
df=df[~df['claim_id'].isin(error_claims)]

### Convert diagnosis code to string and split up diagnosis code values 

In [11]:
df['diagnosis_codes']=df['diagnosis_codes'].astype('str')
df['procedure_code']=df['procedure_code'].astype('str')

In [12]:
s = df['diagnosis_codes'].str.split('^').apply(pd.Series, 1).stack()

s.index = s.index.droplevel(-1) # to line up with df's index

s.name = 'diagnosis_codes' # needs to join

In [13]:
df.drop('diagnosis_codes',inplace=True,axis=1)

In [14]:
df=df.join(s)

In [15]:
df['diagnosis_codes']=df['diagnosis_codes'].astype('str')

#### Remove '.' from diagnosis codes

In [16]:
df['diagnosis_codes'] = df['diagnosis_codes'].map(lambda x: x.translate({ord('.'): None}))

In [17]:
df.head(2)

,patient_id,claim_id,procedure_code,date_service,date_received,diagnosis_codes
0,A1670,1,99999,1/25/2021,1/26/2021,Z01419
0,A1670,1,99999,1/25/2021,1/26/2021,Z1151


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11301 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   patient_id       11301 non-null  object
 1   claim_id         11301 non-null  int64 
 2   procedure_code   11301 non-null  object
 3   date_service     11301 non-null  object
 4   date_received    11301 non-null  object
 5   diagnosis_codes  11301 non-null  object
dtypes: int64(1), object(5)
memory usage: 618.0+ KB


### Check for valid diagnostics and procedure codes

In [19]:
diagnosis_codes=set(df['diagnosis_codes'])
procedure_codes = set(df['procedure_code'])

In [20]:
# looks up valid codes from respecitve files and returns invalid codes if any,else returns empty list
def valid_code(diagnosis_codes,procedure_codes):
    
    valid_diagnosis_codes=set(df_valid_icd_10_codes['code'])
    valid_procedure_codes=set(df_valid_cpt_codes['code'].astype('str'))
    
    invalid_diag_codes=diagnosis_codes-valid_diagnosis_codes
    invalid_proc_codes=procedure_codes-valid_procedure_codes
          
    return invalid_diag_codes,invalid_proc_codes
    

In [21]:
invalid_diag_codes,invalid_proc_codes=valid_code(diagnosis_codes,procedure_codes)

#### Remove invalid procedure and diagnostics code

In [22]:
df=df[~df['diagnosis_codes'].isin(invalid_diag_codes)]

In [23]:
df=df[~df['procedure_code'].isin(invalid_proc_codes)]

### Checking date format and chronology

In [24]:
# return claim_ids with invalid date format
def date_format(date):
    invalid_date_claims=[]
    for index,row in date.iterrows():
        try:
            datetime.datetime.strptime(str(row[1]), '%m/%d/%Y')    
            datetime.datetime.strptime(str(row[2]), '%m/%d/%Y') 
        except ValueError as e:
            invalid_date_claims.append(row[0])
    return invalid_date_claims            

In [25]:
invalid_date_format_claims=date_format(df_dates)

#### Remove claims with invalid dates 

In [26]:
df=df[~df['claim_id'].isin(invalid_date_format_claims)]

In [27]:
#Checks for date chronology, returns claims with invalid chronology
def date_chrono(date):
    invalid_chrono_claims=[]
    for index,row in date.iterrows():
        if datetime.datetime.strptime(str(row[2]), '%m/%d/%Y') < datetime.datetime.strptime(str(row[1]), '%m/%d/%Y') :
            invalid_chrono_claims.append(row[0])
    return invalid_chrono_claims     

In [28]:
df_dates=df_dates[~df_dates['claim_id'].isin(invalid_date_format_claims)]

In [29]:
invalid_chrono_claims=date_chrono(df_dates)

#### Remove claims with invalid chronology

In [30]:
df=df[~df['claim_id'].isin(invalid_chrono_claims)]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6310 entries, 2 to 4999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   patient_id       6310 non-null   object
 1   claim_id         6310 non-null   int64 
 2   procedure_code   6310 non-null   object
 3   date_service     6310 non-null   object
 4   date_received    6310 non-null   object
 5   diagnosis_codes  6310 non-null   object
dtypes: int64(1), object(5)
memory usage: 345.1+ KB


In [32]:
total_file_errors = [error_claims,invalid_diag_codes,invalid_proc_codes,invalid_date_format_claims,invalid_chrono_claims]

In [33]:
for i in range(len(total_file_errors)):    
    if i == 0:
        print(f'There are {len(total_file_errors[i])} claims with a data format that does not match schema definition such as type, missing values or table description')
    elif i == 1:
        print(f'There are {len(total_file_errors[i])} unique invalid diagnosis codes')
    elif i == 2:
        print(f'There are {len(total_file_errors[i])} unique invalid procedure codes')
    elif i == 3:
        print(f'There are {len(total_file_errors[i])} dates with invalid formats')
    elif i == len(total_file_errors)-1:
        print(f'There are {len(total_file_errors[i])} dates with chnronology errors')
        
    

There are 40 claims with a data format that does not match schema definition such as type, missing values or table description
There are 7 unique invalid diagnosis codes
There are 3 unique invalid procedure codes
There are 20 dates with invalid formats
There are 81 dates with chnronology errors


#### Note :The invalid claims have ben filtered out at each check

In [34]:
df['procedure_code'].value_counts().head(10)

88175    317
87591    257
87798    239
87491    234
85018    196
85014    183
85049    173
87799    166
86592    153
82947    152
Name: procedure_code, dtype: int64

In [35]:
df.to_csv('C:/Akhil/USF BAIS/Jobs/Health Verity/clean_sample_claims.csv',index=False)